# The Laplace approximation

### Load packages

In [1]:
from numpy.polynomial.hermite import hermgauss
from matplotlib import pyplot as plt
from numpy.linalg import inv
import pandas as pd
import numpy as np
import scipy
import time
from scipy.stats import norm

### Randomized simulation data for two sites
- Set random seed
- Generate 10 true betas ranged in (-10, 10)
- Generate 2 sigmas for noise variance in different sites
- Generate data X1 and X2 with size (1000, 10)
- Generate result y1 and y2 with bernoulli distibution 

In [2]:
np.random.seed(30)
true_beta = (np.random.rand(10,1) - np.random.rand(10,1)) * 10
true_sigma = np.random.rand(2)
X1 = (np.random.rand(1000, 10) - np.random.rand(1000, 10)) * 10
p1 = 1 / (1 + np.exp(-(X1 @ true_beta + np.random.normal(0, true_sigma[0], 1000).reshape(1000, 1))))
y1 = np.random.binomial(1,p1)
X2 = (np.random.rand(1000, 10) - np.random.rand(1000, 10)) * 10
p2 = 1 / (1 + np.exp(-(X2 @ true_beta + np.random.normal(0, true_sigma[1], 1000).reshape(1000, 1))))
y2 = np.random.binomial(1,p2)

### Definitions

Notation for $\pi_{ij}$:

$$\pi_{ij} = \dfrac{\exp{(X_{ij}^\top\beta_0}+\mu_{i0})}{1 + \exp{(X_{ij}^\top\beta_0}+\mu_{i0})}$$

In [3]:
def Pi(x, beta_0, mu):
    result = np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu))))
    if np.exp(x @ beta_0 + mu).max() == np.inf:
        return np.nan_to_num(result, nan = 1)
    else:
        return result

Notation for $g(\mu_{i0})$:

$$g(\mu_{i0};\beta_0)=\sum_{j=1}^{n_i}\left[y_{ij}\log\pi_{ij}+(1-y_{ij})\log(1-\pi_{ij})\right]+\log\phi(\mu_{i0};\theta_0)$$

In [4]:
def g(x, y, mu, beta_0, tau=1):
    g = sum(y * np.log(Pi(x, beta_0, mu)) + (1 - y) * np.log(1 - Pi(x, beta_0, mu))) \
    + np.log((np.sqrt(2 * np.pi) * tau)**(-1) * np.exp(-mu**2/(2 * tau**2)))
    return g

In [5]:
# Plugged in log-sum-exp trick
def logsumexp(x):
    c = x.max()
    LSE = c + np.log(np.sum(np.exp(x - c)))
    return np.exp(x - LSE)

def g(x, y, mu, beta_0, tau=1):
    g = sum(y * logsumexp(Pi(x, beta_0, mu)) + (1 - y) * logsumexp(1 - Pi(x, beta_0, mu))) \
    + np.log((np.sqrt(2 * np.pi) * tau)**(-1) * np.exp(-mu**2/(2 * tau**2)))
    return g

Notation for $g_\beta(\mu_{i0})$:

$$\dfrac{\partial g}{\partial \beta_0}=\sum_{j=1}^{n_i}(X_{ij}y_{ij}-X_{ij}\pi_{ij})$$

In [6]:
def g_b(x, y, mu, beta_0, tau = 1):
    return np.sum(x * y - x * Pi(x, beta_0, mu), axis = 0)

Notation for $g_\mu(\mu_{i0})$:

$$\dfrac{\partial g}{\partial \mu_{i0}}=\sum_{j=1}^{n_i}(y_{ij}-\pi_{ij})-\dfrac{\mu_{i0}}{\tau_0^2}$$

In [7]:
def g_u(x, y, mu, beta_0, tau = 1):
    return sum(y - Pi(x, beta_0, mu)) - mu/tau**2

Notation for $g_{\mu\mu}(\mu_{i0})$:

$$\dfrac{\partial^2 g}{\partial \mu_{i0}^2}=-\sum_{j=1}^{n_i}\dfrac{\partial\pi_{ij}}{\partial\mu_{i0}}-\dfrac{1}{\tau_0^2}$$

In [8]:
def g_uu(x, y, mu, beta_0, tau = 1):
    result = np.nan_to_num(- np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu))**2)), nan = 0)
    return sum(result) - 1/tau**2

Notation for $g_{\mu\beta}(\mu_{i0})$:

$$\dfrac{\partial^2 g}{\partial \mu_{i0}\partial \beta_0}=-\sum_{j=1}^{n_i}\dfrac{\partial\pi_{ij}}{\partial\beta_0}$$

In [9]:
def g_ub(x, y, mu, beta_0, tau = 1):
    result = np.nan_to_num(np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu))**2)), nan = 0)
    return np.sum(- x * result, axis = 0)

Notation for $g_{\beta\beta}(\mu_{i0})$:

$$\dfrac{\partial^2 g}{\partial \beta_0^2}=-\sum_{j=1}^{n_i}X_{ij}\dfrac{\partial\pi_{ij}}{\partial\beta_0}$$

In [10]:
def g_bb(x, y, mu, beta_0, tau = 1):
    result = 0
    for i in range(len(y)):
        result += -np.asarray(x[i].reshape(x.shape[1],1) @ x[i].reshape(1,x.shape[1])\
        * np.nan_to_num((np.exp(x[i] @ beta_0 + mu) / (1 + np.exp(x[i] @ beta_0 + mu))**2), nan = 0))
    return result

Notation for $g_{\mu\beta}(\hat\mu_{i0};\beta_0)$:

$$\dfrac{\partial g_u(\hat\mu(\beta_0);\beta_0)}{\partial \beta_0}=\hat\mu_\beta(\beta_0)g_{\mu\mu}(\hat\mu_{i0})+g_{\mu\beta}(\hat\mu_{i0})=0$$

Notation for $g_{\mu\mu\mu}(\hat\mu_{i0};\beta_0)$:

$$g_{\mu\mu\mu}(\hat\mu_{i0};\beta_0) = -\sum_{j=1}^{n_i}\dfrac{\partial^2\pi_{ij}}{\partial\mu_{i0}^2}$$

In [11]:
def g_uuu(x, y, mu, beta_0, tau = 1):
    return sum(- np.nan_to_num(np.asarray((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**3)), nan = 0))

Notation for $g_{\mu\mu\beta}(\hat\mu_{i0};\beta_0)$:

$$g_{\mu\mu\beta}(\hat\mu_{i0};\beta_0) = -\sum_{j=1}^{n_i}\dfrac{\partial^2\pi_{ij}}{\partial\mu_{i0}\partial\beta_0}$$

In [12]:
def g_uub(x, y, mu, beta_0, tau = 1):
    return np.sum(- x * np.nan_to_num(np.asarray((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**3)), nan = 0), axis = 0)

Notation for $g_{\mu\beta\beta}(\hat\mu_{i0};\beta_0)$:

$$\dfrac{\partial^3 g}{\partial \mu_{i0}\partial \beta_0^2}=-\sum_{j=1}^{n_i}\dfrac{\partial^2\pi_{ij}}{\partial\beta_0^2}$$

In [13]:
def g_ubb(x, y, mu, beta_0, tau = 1):
    result = 0
    for i in range(len(y)):
        result += -np.asarray(x[i].reshape(x.shape[1],1) @ x[i].reshape(1,x.shape[1])\
                         * np.nan_to_num((np.exp(x[i] @ beta_0 + mu) * (np.exp(x[i] @ beta_0 + mu) - 1)\
                                       / (1 + np.exp(x[i] @ beta_0 + mu))**3), nan = 0))
    return result

Notation for $g_{\mu\mu\mu\mu}(\hat\mu_{i0};\beta_0)$:

$$g_{\mu\mu\mu\mu}(\hat\mu_{i0};\beta_0) = -\sum_{j=1}^{n_i}\dfrac{\partial^3\pi_{ij}}{\partial\mu_{i0}^3}$$

In [14]:
def g_uuuu(x, y, mu, beta_0, tau = 1):
    result = sum(- np.nan_to_num((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**3), nan = 0)\
                 + np.nan_to_num((3 * np.exp(2 * (x @ beta_0 + mu)) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**4), nan = 0)\
                 - np.nan_to_num((np.exp(2 * (x @ beta_0 + mu))  / (1 + np.exp(x @ beta_0 + mu))**3), nan = 0)\
                )
    return result

Notation for $g_{\mu\mu\mu\beta}(\hat\mu_{i0};\beta_0)$:

$$g_{\mu\mu\mu\beta}(\hat\mu_{i0};\beta_0) = -\sum_{j=1}^{n_i}\dfrac{\partial^3\pi_{ij}}{\partial\mu_{i0}^2\partial\beta_0}$$

In [15]:
def g_uuub(x, y, mu, beta_0, tau = 1):
    result = np.sum(- x * np.asarray(np.nan_to_num((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**3), nan = 0)\
                 + np.nan_to_num((3 * np.exp(2 * (x @ beta_0 + mu)) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**4), nan = 0)\
                 - np.nan_to_num((np.exp(2 * (x @ beta_0 + mu))  / (1 + np.exp(x @ beta_0 + mu))**3), nan = 0))
                    , axis = 0)
    return result

Notation for $g_{\mu\mu\beta\beta}(\hat\mu_{i0};\beta_0)$:

$$g_{\mu\mu\beta\beta}(\hat\mu_{i0};\beta_0) = -\sum_{j=1}^{n_i}\dfrac{\partial^3\pi_{ij}}{\partial\mu_{i0}\partial\beta^2_0}$$

In [16]:
def g_uubb(x, y, mu, beta_0, tau = 1):
    result = 0
    for i in range(len(y)):
        result += -np.asarray(x[i].reshape(x.shape[1],1) @ x[i].reshape(1,x.shape[1])\
                             * (np.nan_to_num((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**3), nan = 0)\
                 + np.nan_to_num((3 * np.exp(2 * (x @ beta_0 + mu)) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**4), nan = 0)\
                 - np.nan_to_num((np.exp(2 * (x @ beta_0 + mu))  / (1 + np.exp(x @ beta_0 + mu))**3), nan = 0)))
        
    return result

Notation for $\hat\omega$:

$$\hat\omega=\sqrt{-\dfrac{1}{g_{\mu\mu}(\hat\mu_{i0})}}$$

In [17]:
def omega(x, y, mu, beta_0, tau = 1):
    return np.sqrt(-1/g_uu(x, y, mu, beta_0))

Notation for $\hat\omega_\beta$:

$$\dfrac{\partial\hat\omega}{\partial\beta}=\dfrac{1}{2}\hat\omega^3(g_{\mu\mu\mu}\hat\mu_\beta + g_{\mu\mu\beta})$$

In [18]:
def omega_b(x, y, mu, beta_0, tau = 1):
    return 0.5 * omega(x, y, mu, beta_0, tau)**3 * (g_uuu(x, y, mu, beta_0, tau) * mu_b(x, y, mu, beta_0, tau)\
                                                    + g_uub(x, y, mu, beta_0, tau))

Notation for $\hat\mu_\beta$

$$\hat\mu_\beta(\beta_0)=-\dfrac{g_{\mu\beta}(\hat\mu_{i0};\beta_0)}{g_{\mu\mu}(\hat\mu_{i0};\beta_0)}=\hat\omega^2(\beta_0)g_{\mu\beta}(\hat\mu_{i0}(\beta_0);\beta_0)$$

In [19]:
def mu_b(x, y, mu, beta_0, tau = 1):
    return omega(x, y, mu, beta_0, tau)**2 * g_ub(x, y, mu, beta_0, tau)

Notation for $\hat\mu_{\beta\beta'}$:

$$\hat\mu_{\beta\beta'}(\beta_0)=\hat\omega^2(\hat\mu_\beta\hat\mu_{\beta'} g_{\mu\mu\mu}+\hat\mu_{\beta}g_{\mu\mu\beta'}+\hat\mu_{\beta'}g_{\mu\mu\beta}+g_{\mu\beta\beta'})$$

In [20]:
def mu_bb(x, y, mu, beta_0, tau = 1):
    result = omega(x, y, mu, beta_0, tau)**2 * (mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1],1)\
                                              @ mu_b(x, y, mu, beta_0, tau).reshape(1,x.shape[1])\
                                              * g_uuu(x, y, mu, beta_0, tau)\
                                              + 2 * mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1],1)\
                                              @ g_uub(x, y, mu, beta_0, tau).reshape(1,x.shape[1])\
                                              + g_ubb(x, y, mu, beta_0, tau))
    return result

Notation for $\hat\omega_{\beta\beta'}$:

$$\dfrac{\partial^2}{\partial\beta\partial\beta'}\hat\omega(\beta_0)=\frac{3}{4}\hat\omega^5(\hat\mu_{\beta'}g_{\mu\mu\mu}+g_{\mu\mu\beta'})(\hat\mu_{\beta}g_{\mu\mu\mu}+g_{\mu\mu\beta})+\dfrac{1}{2}\hat\omega^3(\hat\mu_{\beta\beta'}g_{\mu\mu\mu}+\hat\mu_\beta\hat\mu_{\beta'}g_{\mu\mu\mu\mu}+\hat\mu_{\beta}g_{\mu\mu\mu\beta'}+\hat\mu_{\beta\beta'}g_{\mu\mu\mu\beta}+g_{\mu\mu\beta\beta'})$$

In [21]:
def omega_bb(x, y, mu, beta_0, tau = 1):
    result = 3/4 * omega(x, y, mu, beta_0, tau)**5 * (mu_b(x, y, mu, beta_0, tau) * g_uuu(x, y, mu, beta_0, tau)\
                                                      + g_uub(x, y, mu, beta_0, tau)).reshape(x.shape[1],1)\
     @ (mu_b(x, y, mu, beta_0, tau) * g_uuu(x, y, mu, beta_0, tau)\
        + g_uub(x, y, mu, beta_0, tau)).reshape(1,x.shape[1]) + 1/2 * omega(x, y, mu, beta_0, tau)**3\
     * (mu_bb(x, y, mu, beta_0, tau) * g_uuu(x, y, mu, beta_0, tau)\
        + (mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1],1)\
           @ mu_b(x, y, mu, beta_0, tau).reshape(1,x.shape[1]) * g_uuuu(x, y, mu, beta_0, tau))\
        + mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1],1) @ g_uuub(x, y, mu, beta_0, tau).reshape(1,x.shape[1])\
        + mu_bb(x, y, mu, beta_0, tau)
       )
    return result

Notation for $\mathcal l_i'$:

$$\mathcal l_i' = \dfrac{\partial\mathcal l_i}{\partial\beta_0}=\dfrac{\hat\omega_\beta}{\hat\omega}+g_\mu(\hat\mu_{i0};\beta_0)\hat\mu_\beta(\beta_0)+g_\beta(\hat\mu_{i0};\beta_0)$$

In [22]:
def l_1(x, y, mu, beta_0, tau = 1):
    l1 = omega_b(x, y, mu, beta_0, tau) / omega(x, y, mu, beta_0, tau)\
    + g_u(x, y, mu, beta_0, tau) * mu_b(x, y, mu, beta_0, tau) + g_b(x, y, mu, beta_0, tau)
    return l1

Notation for $l''_i$:

$$l''_i=\dfrac{\partial^2l_i}{\partial\beta_0^2}=\dfrac{\hat\omega_{\beta\beta'}\hat\omega-\hat\omega_\beta\hat\omega_{\beta'}}{\hat\omega^2}+\hat\mu_{\beta\beta'}g_\mu+\hat\mu_\beta(\hat\mu_{\beta'}g_{\mu\mu}+g_{\mu\beta'})+\hat\mu_{\beta'}g_{\mu\beta}+g_{\beta\beta'}$$

In [23]:
def l_2(x, y, mu, beta_0, tau = 1):
    l2 = omega(x, y, mu, beta_0, tau)**(-2) * (omega_bb(x, y, mu, beta_0, tau) * omega(x, y, mu, beta_0, tau)\
                                               - omega_b(x, y, mu, beta_0, tau).reshape(x.shape[1],1)\
                                               @ omega_b(x, y, mu, beta_0, tau).reshape(1,x.shape[1]))\
     + mu_bb(x, y, mu, beta_0, tau) * g_u(x, y, mu, beta_0, tau) + mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1],1)\
     @ (mu_b(x, y, mu, beta_0, tau) * g_uu(x, y, mu, beta_0, tau) + g_ub(x, y, mu, beta_0, tau)).reshape(1,x.shape[1])\
     + mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1],1) @ g_ub(x, y, mu, beta_0, tau).reshape(1,x.shape[1])\
     + g_bb(x, y, mu, beta_0, tau)
    return l2

Notation for maximizing $\mu$ with respect to $\hat\mu=\arg\max_\mu g(\mu_{i0};\beta_0)$

In [24]:
def max_mu(x, y, mu, beta_0, tau=1, max_iter=100):
    for step in range(max_iter):
#         print('Step: ', step, '\n')
        mu_new = mu - g_u(x, y, mu, beta_0, tau)/g_uu(x, y, mu, beta_0, tau)
        diff = mu_new - mu
#         print(diff)
        if np.abs(diff) < 10**(-10):
#             print(mu)
            break;
        mu = mu_new
    return mu

Notation for updating parameters

In [25]:
beta = np.repeat(0, 10).reshape(10,1)
mu = [0.1, 0.1]
tau = 1

start_time = time.time()
print('Initial beta:', beta, "\n")
for step_mu in range(1):
    mu[0] = max_mu(X1, y1, mu[0], beta, tau)
    mu[1] = max_mu(X2, y2, mu[1], beta, tau)
    for step in range(100):
        l1 = l_1(X1, y1, mu[0], beta, tau) + l_1(X2, y2, mu[1], beta, tau)
        l2 = l_2(X1, y1, mu[0], beta, tau) + l_2(X2, y2, mu[1], beta, tau)
        delta = l1 @ inv(l2)
        new_beta = beta - delta.reshape(10,1)
        if max(np.abs(delta)) < 10**(-10):
            break;
        beta = new_beta
        print('Step ', step+1, ':\n')
        print('Beta:\n', beta, '\n')
        print('Diff:\n', delta, '\n')
print('Beta:\n', beta, '\n')
print("--- %s seconds ---" % (time.time() - start_time))

Initial beta: [[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]] 

Step  1 :

Beta:
 [[ 0.13358096]
 [-0.04227759]
 [ 0.03450702]
 [-0.16768401]
 [ 0.00319927]
 [ 0.05814861]
 [ 0.21503741]
 [-0.20798613]
 [ 0.09450034]
 [ 0.05144011]] 

Diff:
 [-0.13358096  0.04227759 -0.03450702  0.16768401 -0.00319927 -0.05814861
 -0.21503741  0.20798613 -0.09450034 -0.05144011] 

Step  2 :

Beta:
 [[ 0.25236852]
 [-0.0815835 ]
 [ 0.06716006]
 [-0.32319658]
 [ 0.00725054]
 [ 0.10916495]
 [ 0.40427813]
 [-0.3927978 ]
 [ 0.17913935]
 [ 0.08974294]] 

Diff:
 [-0.11878757  0.03930591 -0.03265304  0.15551257 -0.00405127 -0.05101634
 -0.18924072  0.18481167 -0.08463902 -0.03830283] 

Step  3 :

Beta:
 [[ 0.41199792]
 [-0.13325206]
 [ 0.11076937]
 [-0.53104427]
 [ 0.01732386]
 [ 0.17994442]
 [ 0.65848616]
 [-0.63979824]
 [ 0.29297944]
 [ 0.13956991]] 

Diff:
 [-0.1596294   0.05166856 -0.04360931  0.20784769 -0.01007332 -0.07077947
 -0.25420804  0.24700045 -0.11384008 -0.04982697] 

Step  4 :

Beta:
 [[ 0.6

In [175]:
true_beta

array([[ 5.07909214],
       [-1.63387798],
       [ 1.44871559],
       [-6.0320438 ],
       [ 0.2875767 ],
       [ 2.56958458],
       [ 7.95979736],
       [-7.59135789],
       [ 3.50513771],
       [ 1.6770373 ]])

In [176]:
beta

array([[ 4.85637366],
       [-1.54311161],
       [ 1.37036597],
       [-5.80211922],
       [ 0.26036163],
       [ 2.36853001],
       [ 7.55281715],
       [-7.24942762],
       [ 3.33391189],
       [ 1.54162541]])

In [26]:
true_beta - beta

array([[ 0.23342346],
       [-0.09434282],
       [ 0.08182335],
       [-0.24307494],
       [ 0.02780038],
       [ 0.20624836],
       [ 0.42427024],
       [-0.35844645],
       [ 0.17862849],
       [ 0.13897912]])

# Significance of params

In [25]:
X = np.concatenate([X1, X2])

V = np.diagflat(Pi(X, beta, 0) * (1 - Pi(X, beta, 0)))

SE = np.sqrt(np.diag(inv(np.transpose(X) @ V @ X))).reshape(10,1)

Z = beta/SE

P = 2 * norm.cdf(-1 * np.abs(Z))

CI_025  = beta - 1.959964 * SE
CI_975  = beta + 1.959964 * SE

df = pd.DataFrame({'Truth': np.transpose(true_beta)[0], 'Coef': np.transpose(beta)[0], 'Std.Err': np.transpose(SE)[0], 
                   'z': np.transpose(Z)[0], 'P-value': np.transpose(P)[0], 
                   '[0.025': np.transpose(CI_025)[0], '0.975]': np.transpose(CI_975)[0]},
                  index = col)

NameError: name 'beta' is not defined

In [154]:
df #Estimated result (Laplace approximation)

,Truth,Coef,Std.Err,z,P-value,[0.025,0.975]
V1,5.079092,4.845669,0.736134,6.582588,4.623301e-11,3.402872,6.288465
V2,-1.633878,-1.539535,0.245113,-6.280920,3.365746e-10,-2.019948,-1.059123
V3,1.448716,1.366892,0.223359,6.119705,9.374850e-10,0.929116,1.804668
V4,-6.032044,-5.788969,0.863616,-6.703176,2.039373e-11,-7.481625,-4.096313
V5,0.287577,0.259776,0.088533,2.934219,3.343880e-03,0.086254,0.433299
V6,2.569585,2.363336,0.370422,6.380112,1.769581e-10,1.637322,3.089351
V7,7.959797,7.535527,1.126965,6.686565,2.284704e-11,5.326715,9.744339
V8,-7.591358,-7.232911,1.084795,-6.667536,2.601333e-11,-9.359071,-5.106752
V9,3.505138,3.326509,0.501654,6.631078,3.332441e-11,2.343285,4.309734
V10,1.677037,1.538058,0.243683,6.311710,2.759696e-10,1.060448,2.015669


# stable test

In [161]:
dif_ = []
for seed_index in range(30):
    np.random.seed(seed_index)
    true_beta = (np.random.rand(10,1) - np.random.rand(10,1)) * 10
    true_sigma = np.random.rand(2)
    X1 = (np.random.rand(1000, 10) - np.random.rand(1000, 10)) * 10
    p1 = 1 / (1 + np.exp(-(X1 @ true_beta + np.random.normal(0, true_sigma[0], 1000).reshape(1000, 1))))
    y1 = np.random.binomial(1,p1)
    X2 = (np.random.rand(1000, 10) - np.random.rand(1000, 10)) * 10
    p2 = 1 / (1 + np.exp(-(X2 @ true_beta + np.random.normal(0, true_sigma[1], 1000).reshape(1000, 1))))
    y2 = np.random.binomial(1,p2)

    beta = np.repeat(0, 10).reshape(10,1)
    mu = [0.1, 0.1]
    tau = 1

    start_time = time.time()
#     print('Initial beta:', beta, "\n")
    for step_mu in range(1):
        mu[0] = max_mu(X1, y1, mu[0], beta, tau)
        mu[1] = max_mu(X2, y2, mu[1], beta, tau)
        for step in range(100):
            l1 = l_1(X1, y1, mu[0], beta, tau) + l_1(X2, y2, mu[1], beta, tau)
            l2 = l_2(X1, y1, mu[0], beta, tau) + l_2(X2, y2, mu[1], beta, tau)
            delta = l1 @ inv(l2)
            new_beta = beta - delta.reshape(10,1)
            if max(np.abs(delta)) < 10**(-10):
                break;
            beta = new_beta
    #         print('Step ', step+1, ':\n')
    #         print('Beta:\n', beta, '\n')
    #         print('Diff:\n', delta, '\n')
    # print('Beta:\n', beta, '\n')
    print("--- %s seconds ---" % (time.time() - start_time))
    dif_ += [true_beta - beta]

--- 0.2714729309082031 seconds ---
--- 0.25629281997680664 seconds ---
--- 0.25200390815734863 seconds ---
--- 0.256655216217041 seconds ---
--- 0.2698948383331299 seconds ---
--- 0.25968313217163086 seconds ---
--- 0.2565948963165283 seconds ---
--- 0.2627601623535156 seconds ---
--- 0.2559070587158203 seconds ---
--- 0.251723051071167 seconds ---
--- 0.25339508056640625 seconds ---
--- 0.250715970993042 seconds ---
--- 0.2512221336364746 seconds ---
--- 0.2543320655822754 seconds ---
--- 0.2521059513092041 seconds ---
--- 0.2511448860168457 seconds ---
--- 0.252871036529541 seconds ---
--- 0.2520027160644531 seconds ---
--- 0.2644810676574707 seconds ---
--- 0.2631680965423584 seconds ---
--- 0.25179076194763184 seconds ---
--- 0.25327610969543457 seconds ---
--- 0.2727019786834717 seconds ---
--- 0.26079583168029785 seconds ---
--- 0.2533707618713379 seconds ---
--- 0.2513909339904785 seconds ---
--- 0.2552368640899658 seconds ---
--- 0.2570350170135498 seconds ---
--- 0.26158905029

In [180]:
col

['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10']

# Simulation with Penn

In [26]:
import os
truth = np.array([-1.5,0.1,-0.5,-0.3,0.4,-0.2,-0.25,0.35,-0.1,0.5]).reshape(10, 1)
var_name = []
for i in range(10):
    var_name += ['X' + str(i+1)]

## Setting 1

In [27]:
def LA(X1, y1, X2, y2):
    beta = np.repeat(0, 10).reshape(10,1)
    mu = [0.1, 0.1]
    tau = 1

#     start_time = time.time()
#     print('Initial beta:', beta, "\n")
    for step_mu in range(3):
        mu[0] = max_mu(X1, y1, mu[0], beta, tau)
        mu[1] = max_mu(X2, y2, mu[1], beta, tau)
        for step in range(100):
            l1 = l_1(X1, y1, mu[0], beta, tau) + l_1(X2, y2, mu[1], beta, tau)
            l2 = l_2(X1, y1, mu[0], beta, tau) + l_2(X2, y2, mu[1], beta, tau)
            delta = l1 @ inv(l2)
            new_beta = beta - delta.reshape(10,1)
            if max(np.abs(delta)) < 10**(-10):
                break;
            beta = new_beta
            if True in np.isnan(beta):
                break;
#             print('Step ', step+1, ':\n')
#             print('Beta:\n', beta, '\n')
#             print('Diff:\n', delta, '\n')
        if True in np.isnan(beta):
                break;
#     print('Beta:\n', beta, '\n')
#     print("--- %s seconds ---" % (time.time() - start_time))
    return [beta, mu]

In [28]:
def out(X1, X2, beta, true_beta):
    X = np.concatenate([X1, X2])

    V = np.diagflat(Pi(X, beta, 0) * (1 - Pi(X, beta, 0)))

    SE = np.sqrt(np.diag(inv(np.transpose(X) @ V @ X))).reshape(10,1)

    Z = beta/SE

    P = 2 * norm.cdf(-1 * np.abs(Z))

    CI_025  = beta - 1.959964 * SE
    CI_975  = beta + 1.959964 * SE

    df = pd.DataFrame({'Truth': np.transpose(true_beta)[0], 'Coef': np.transpose(beta)[0], 'Std.Err': np.transpose(SE)[0], 
                       'z': np.transpose(Z)[0], 'P-value': np.transpose(P)[0], 
                       '[0.025': np.transpose(CI_025)[0], '0.975]': np.transpose(CI_975)[0]},
                      index = var_name)
    return df

### Data loading

In [29]:
file_dir = '../Simulation_data_GLMM/Setting_1/'
file_names = os.listdir(file_dir)

### 2 Sites Simulation

In [30]:
for i in range(len(file_names)):
    try:
        start_time = time.time()
        df = pd.read_csv(file_dir + file_names[i], index_col=0)
        # Load data
        data1 = np.array(df[df['Site_ID'] == 1][var_name])
        data2 = np.array(df[df['Site_ID'] == 2][var_name])
        out1 = np.array(df[df['Site_ID'] == 1]['y']).reshape(500,1)
        out2 = np.array(df[df['Site_ID'] == 2]['y']).reshape(500,1)
        # Simulations
        [beta, mu] = LA(data1, out1, data2, out2)
        out(data1, data2, beta, truth).to_csv('../Simulation_data_GLMM/Result/Setting_1_' + 
                                              file_names[i][44:], header = True)
        print('======\n File:\n', file_names[i], '\n Completed!\n')
        print("--- %s seconds ---" % (time.time() - start_time))
    except:
        pass;

 File:
 2Sites_500PatientsEachSite_SmallVar1_Dataset4.csv 
 Completed!

--- 43.39169502258301 seconds ---


/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in exp
  
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  
/Users/liwentao/opt/ana

 File:
 2Sites_500PatientsEachSite_SmallVar1_Dataset7.csv 
 Completed!

--- 19.697643041610718 seconds ---


/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in square
  
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in multiply
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in power
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in multiply
  
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in power
  
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in square
  """


 File:
 2Sites_500PatientsEachSite_SmallVar1_Dataset2.csv 
 Completed!

--- 43.08603024482727 seconds ---
 File:
 2Sites_500PatientsEachSite_SmallVar1_Dataset3.csv 
 Completed!

--- 14.073592185974121 seconds ---
 File:
 2Sites_500PatientsEachSite_SmallVar1_Dataset20.csv 
 Completed!

--- 15.778317213058472 seconds ---
 File:
 2Sites_500PatientsEachSite_SmallVar1_Dataset13.csv 
 Completed!

--- 46.239689111709595 seconds ---
 File:
 2Sites_500PatientsEachSite_SmallVar1_Dataset11.csv 
 Completed!

--- 15.381375074386597 seconds ---
 File:
 2Sites_500PatientsEachSite_SmallVar1_Dataset17.csv 
 Completed!

--- 42.55866503715515 seconds ---
 File:
 2Sites_500PatientsEachSite_SmallVar1_Dataset8.csv 
 Completed!

--- 41.72289705276489 seconds ---
 File:
 2Sites_500PatientsEachSite_SmallVar1_Dataset9.csv 
 Completed!

--- 12.386541843414307 seconds ---


# Setting 2

### Data loading

In [31]:
file_dir = '../Simulation_data_GLMM/Setting_2/'
file_names = os.listdir(file_dir)

### 2 Sites Simulation

In [32]:
for i in range(len(file_names)):
    try:
        start_time = time.time()
        df = pd.read_csv(file_dir + file_names[i], index_col=0)
        # Load data
        data1 = np.array(df[df['Site_ID'] == 1][var_name])
        data2 = np.array(df[df['Site_ID'] == 2][var_name])
        out1 = np.array(df[df['Site_ID'] == 1]['y']).reshape(500,1)
        out2 = np.array(df[df['Site_ID'] == 2]['y']).reshape(500,1)
        # Simulations
        [beta, mu] = LA(data1, out1, data2, out2)
        out(data1, data2, beta, truth).to_csv('../Simulation_data_GLMM/Result/Setting_2_' + 
                                              file_names[i][44:], header = True)
        print('======\n File:\n', file_names[i], '\n Completed!\n')
        print("--- %s seconds ---" % (time.time() - start_time))
    except:
        pass;

 File:
 2Sites_500PatientsEachSite_LargeVar4_Dataset2.csv 
 Completed!

--- 48.52385997772217 seconds ---
 File:
 2Sites_500PatientsEachSite_LargeVar4_Dataset16.csv 
 Completed!

--- 42.70333003997803 seconds ---
 File:
 2Sites_500PatientsEachSite_LargeVar4_Dataset17.csv 
 Completed!

--- 42.725252866744995 seconds ---
 File:
 2Sites_500PatientsEachSite_LargeVar4_Dataset3.csv 
 Completed!

--- 10.934883117675781 seconds ---


/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in square
  
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in multiply
  This is separate from the ipy

 File:
 2Sites_500PatientsEachSite_LargeVar4_Dataset4.csv 
 Completed!

--- 50.630802154541016 seconds ---
 File:
 2Sites_500PatientsEachSite_LargeVar4_Dataset11.csv 
 Completed!

--- 12.616214990615845 seconds ---
 File:
 2Sites_500PatientsEachSite_LargeVar4_Dataset7.csv 
 Completed!

--- 17.509664058685303 seconds ---
 File:
 2Sites_500PatientsEachSite_LargeVar4_Dataset13.csv 
 Completed!

--- 24.87748670578003 seconds ---
 File:
 2Sites_500PatientsEachSite_LargeVar4_Dataset8.csv 
 Completed!

--- 25.97940993309021 seconds ---
 File:
 2Sites_500PatientsEachSite_LargeVar4_Dataset20.csv 
 Completed!

--- 14.248498916625977 seconds ---
 File:
 2Sites_500PatientsEachSite_LargeVar4_Dataset9.csv 
 Completed!

--- 13.808903932571411 seconds ---
 File:
 2Sites_500PatientsEachSite_LargeVar4_Dataset18.csv 
 Completed!

--- 52.115232944488525 seconds ---


# Setting 5

### Data loading

In [392]:
file_dir = '../Simulation_data_GLMM/Setting_5/'
file_names = os.listdir(file_dir)

### 2 Sites Simulation

In [393]:
for i in range(len(file_names)):
    try:
        start_time = time.time()
        df = pd.read_csv(file_dir + file_names[i], index_col=0)
        # Load data
        data1 = np.array(df[df['Site_ID'] == 1][var_name])
        data2 = np.array(df[df['Site_ID'] == 2][var_name])
        out1 = np.array(df[df['Site_ID'] == 1]['y']).reshape(30,1)
        out2 = np.array(df[df['Site_ID'] == 2]['y']).reshape(30,1)
        # Simulations
        [beta, mu] = LA(data1, out1, data2, out2)
        out(data1, data2, beta, truth).to_csv('../Simulation_data_GLMM/Result/Setting_5_' + 
                                              file_names[i][43:], header = True)
        print('======\n File:\n', file_names[i], '\n Completed!\n')
        print("--- %s seconds ---" % (time.time() - start_time))
    except:
        pass;

/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in square
  
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in multiply
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in power
  This is separate from the ipykernel pack

 File:
 2Sites_30PatientsEachSite_SmallVar1_Dataset5.csv 
 Completed!

--- 0.1976151466369629 seconds ---
 File:
 2Sites_30PatientsEachSite_SmallVar1_Dataset4.csv 
 Completed!

--- 0.12484407424926758 seconds ---
 File:
 2Sites_30PatientsEachSite_SmallVar1_Dataset6.csv 
 Completed!

--- 7.298659086227417 seconds ---
 File:
 2Sites_30PatientsEachSite_SmallVar1_Dataset7.csv 
 Completed!

--- 0.12478399276733398 seconds ---
 File:
 2Sites_30PatientsEachSite_SmallVar1_Dataset3.csv 
 Completed!

--- 0.13857197761535645 seconds ---
 File:
 2Sites_30PatientsEachSite_SmallVar1_Dataset2.csv 
 Completed!

--- 8.185570001602173 seconds ---
 File:
 2Sites_30PatientsEachSite_SmallVar1_Dataset1.csv 
 Completed!

--- 0.2390739917755127 seconds ---
 File:
 2Sites_30PatientsEachSite_SmallVar1_Dataset19.csv 
 Completed!

--- 4.16333794593811 seconds ---
 File:
 2Sites_30PatientsEachSite_SmallVar1_Dataset18.csv 
 Completed!

--- 0.1957247257232666 seconds ---
 File:
 2Sites_30PatientsEachSite_SmallVar1_D

# Setting 6

### Data loading

In [395]:
file_dir = '../Simulation_data_GLMM/Setting_6/'
file_names = os.listdir(file_dir)

### 2 Sites Simulation

In [396]:
for i in range(len(file_names)):
    try:
        start_time = time.time()
        df = pd.read_csv(file_dir + file_names[i], index_col=0)
        # Load data
        data1 = np.array(df[df['Site_ID'] == 1][var_name])
        data2 = np.array(df[df['Site_ID'] == 2][var_name])
        out1 = np.array(df[df['Site_ID'] == 1]['y']).reshape(30,1)
        out2 = np.array(df[df['Site_ID'] == 2]['y']).reshape(30,1)
        # Simulations
        [beta, mu] = LA(data1, out1, data2, out2)
        out(data1, data2, beta, truth).to_csv('../Simulation_data_GLMM/Result/Setting_6_' + 
                                              file_names[i][43:], header = True)
        print('======\n File:\n', file_names[i], '\n Completed!\n')
        print("--- %s seconds ---" % (time.time() - start_time))
    except:
        pass;

/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in power
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in power
  
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in multiply
  
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in square
  
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in multiply
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/liwentao/opt/anaconda3/li

 File:
 2Sites_30PatientsEachSite_LargeVar4_Dataset17.csv 
 Completed!

--- 0.17728304862976074 seconds ---
 File:
 2Sites_30PatientsEachSite_LargeVar4_Dataset3.csv 
 Completed!

--- 0.18227005004882812 seconds ---
 File:
 2Sites_30PatientsEachSite_LargeVar4_Dataset2.csv 
 Completed!

--- 7.61321496963501 seconds ---
 File:
 2Sites_30PatientsEachSite_LargeVar4_Dataset16.csv 
 Completed!

--- 0.15086007118225098 seconds ---


/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in exp
  """
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


 File:
 2Sites_30PatientsEachSite_LargeVar4_Dataset14.csv 
 Completed!

--- 0.7621369361877441 seconds ---
 File:
 2Sites_30PatientsEachSite_LargeVar4_Dataset1.csv 
 Completed!

--- 0.2309877872467041 seconds ---
 File:
 2Sites_30PatientsEachSite_LargeVar4_Dataset15.csv 
 Completed!

--- 0.15584897994995117 seconds ---
 File:
 2Sites_30PatientsEachSite_LargeVar4_Dataset11.csv 
 Completed!

--- 0.15219688415527344 seconds ---
 File:
 2Sites_30PatientsEachSite_LargeVar4_Dataset5.csv 
 Completed!

--- 3.6684858798980713 seconds ---
 File:
 2Sites_30PatientsEachSite_LargeVar4_Dataset4.csv 
 Completed!

--- 0.1391749382019043 seconds ---
 File:
 2Sites_30PatientsEachSite_LargeVar4_Dataset10.csv 
 Completed!

--- 0.18236398696899414 seconds ---
 File:
 2Sites_30PatientsEachSite_LargeVar4_Dataset12.csv 
 Completed!

--- 0.20760393142700195 seconds ---
 File:
 2Sites_30PatientsEachSite_LargeVar4_Dataset6.csv 
 Completed!

--- 9.587594032287598 seconds ---
 File:
 2Sites_30PatientsEachSite_Larg

# Setting 3

In [468]:
def LA(X, y):
    beta = np.repeat(0, 10).reshape(10,1)
    mu = np.repeat(0.1, len(y))
    tau = 1

    start_time = time.time()
    print('Initial beta:', beta, "\n")
    for step_mu in range(3):
        for i in range(len(mu)):
            mu[i] = max_mu(X[i], y[i], mu[i], beta, tau)
        for step in range(100):
            l1 = 0
            l2 = 0
            for i in range(len(mu)):
                l1 += l_1(X[i], y[i], mu[i], beta, tau)
                l2 += l_2(X[i], y[i], mu[i], beta, tau)
            delta = l1 @ inv(l2)
            new_beta = beta - delta.reshape(10,1)
            if max(np.abs(delta)) < 10**(-10):
                break;
            beta = new_beta
            if True in np.isnan(beta):
                break;
            print('Step ', step+1, ':\n')
            print('Beta:\n', beta, '\n')
            print('Diff:\n', delta, '\n')
        if True in np.isnan(beta):
                break;
    print('Beta:\n', beta, '\n')
    print("--- %s seconds ---" % (time.time() - start_time))
    return [beta, mu]

In [443]:
def out(X, beta, true_beta):
    X = np.concatenate(X)

    V = np.diagflat(Pi(X, beta, 0) * (1 - Pi(X, beta, 0)))

    SE = np.sqrt(np.diag(inv(np.transpose(X) @ V @ X))).reshape(10,1)

    Z = beta/SE

    P = 2 * norm.cdf(-1 * np.abs(Z))

    CI_025  = beta - 1.959964 * SE
    CI_975  = beta + 1.959964 * SE

    df = pd.DataFrame({'Truth': np.transpose(true_beta)[0], 'Coef': np.transpose(beta)[0], 'Std.Err': np.transpose(SE)[0], 
                       'z': np.transpose(Z)[0], 'P-value': np.transpose(P)[0], 
                       '[0.025': np.transpose(CI_025)[0], '0.975]': np.transpose(CI_975)[0]},
                      index = var_name)
    return df

### Data loading

In [472]:
file_dir = '../Simulation_data_GLMM/Setting_3/'
file_names = os.listdir(file_dir)

### 10 Sites Simulation

In [463]:
for i in range(len(file_names)):
    try:
        start_time = time.time()
        df = pd.read_csv(file_dir + file_names[i], index_col=0)
#         Load data
        data0 = []
        for k in range(10):
            globals()['data%s' % str(k+1)] = np.array(df[df['Site_ID'] == k+1][var_name])
            data0.append(globals()['data%s' % str(k+1)])
        out0 = []
        for k in range(10):
            globals()['out%s' % str(k+1)] = np.array(df[df['Site_ID'] == k+1]['y']).reshape(500,1)
            out0.append(globals()['out%s' % str(k+1)])
        # Simulations
        [beta, mu] = LA(data0, out0)
        out(data0, beta, truth).to_csv('../Simulation_data_GLMM/Result/Setting_3_' + 
                                              file_names[i][45:], header = True)
        print('======\n File:\n', file_names[i], '\n Completed!\n')
        print("--- %s seconds ---" % (time.time() - start_time))
    except:
        pass;

/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in power
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: Ru

 File:
 10Sites_500PatientsEachSite_SmallVar1_Dataset1.csv 
 Completed!

--- 40.187448024749756 seconds ---
 File:
 10Sites_500PatientsEachSite_SmallVar1_Dataset18.csv 
 Completed!

--- 20.52224373817444 seconds ---


/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in square
  
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in multiply
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/liwentao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in square
  """


 File:
 10Sites_500PatientsEachSite_SmallVar1_Dataset2.csv 
 Completed!

--- 29.42949628829956 seconds ---
 File:
 10Sites_500PatientsEachSite_SmallVar1_Dataset3.csv 
 Completed!

--- 14.232635021209717 seconds ---
 File:
 10Sites_500PatientsEachSite_SmallVar1_Dataset19.csv 
 Completed!

--- 19.0919086933136 seconds ---
 File:
 10Sites_500PatientsEachSite_SmallVar1_Dataset7.csv 
 Completed!

--- 40.967844009399414 seconds ---
 File:
 10Sites_500PatientsEachSite_SmallVar1_Dataset6.csv 
 Completed!

--- 17.542230129241943 seconds ---
 File:
 10Sites_500PatientsEachSite_SmallVar1_Dataset4.csv 
 Completed!

--- 13.323127746582031 seconds ---
 File:
 10Sites_500PatientsEachSite_SmallVar1_Dataset8.csv 
 Completed!

--- 16.960466861724854 seconds ---
 File:
 10Sites_500PatientsEachSite_SmallVar1_Dataset12.csv 
 Completed!

--- 35.7163610458374 seconds ---
 File:
 10Sites_500PatientsEachSite_SmallVar1_Dataset13.csv 
 Completed!

--- 13.818329811096191 seconds ---
 File:
 10Sites_500PatientsEac

In [410]:
np.concatenate([data1, data2]).shape

(60, 10)

In [470]:
df = pd.read_csv(file_dir + file_names[0], index_col=0)
    # Load data
data0 = []
for k in range(2):
    globals()['data%s' % str(k+1)] = np.array(df[df['Site_ID'] == k+1][var_name])
    data0.append(globals()['data%s' % str(k+1)])
out0 = []
for k in range(2):
    globals()['out%s' % str(k+1)] = np.array(df[df['Site_ID'] == k+1]['y']).reshape(500,1)
    out0.append(globals()['out%s' % str(k+1)])

In [460]:
max_mu(data0[0], out0[0], mu[0], beta)

array([-1.67860161])

In [471]:
LA(data0, out0)

Initial beta: [[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]] 

Step  1 :

Beta:
 [[-0.23233179]
 [ 0.07168454]
 [-0.27671034]
 [-0.1814745 ]
 [ 0.27102154]
 [-0.22350657]
 [-0.20550898]
 [ 0.24510621]
 [-0.2639585 ]
 [ 0.50722567]] 

Diff:
 [ 0.23233179 -0.07168454  0.27671034  0.1814745  -0.27102154  0.22350657
  0.20550898 -0.24510621  0.2639585  -0.50722567] 

Step  2 :

Beta:
 [[-0.43753286]
 [ 0.06958267]
 [-0.3010379 ]
 [-0.18211167]
 [ 0.26291998]
 [-0.2264583 ]
 [-0.2092315 ]
 [ 0.26252272]
 [-0.28466727]
 [ 0.5085499 ]] 

Diff:
 [ 0.20520107  0.00210186  0.02432755  0.00063716  0.00810156  0.00295173
  0.00372252 -0.01741651  0.02070877 -0.00132423] 

Step  3 :

Beta:
 [[-0.41392801]
 [ 0.0705956 ]
 [-0.28490488]
 [-0.18198215]
 [ 0.26747436]
 [-0.2244435 ]
 [-0.20666865]
 [ 0.25176179]
 [-0.27190589]
 [ 0.50749698]] 

Diff:
 [-0.02360485 -0.00101293 -0.01613302 -0.00012951 -0.00455438 -0.0020148
 -0.00256285  0.01076093 -0.01276138  0.00105293] 

Step  4 :

Beta:
 [[-0.44

Step  28 :

Beta:
 [[-0.44445593]
 [ 0.06903062]
 [-0.31350196]
 [-0.17732215]
 [ 0.26137614]
 [-0.22504247]
 [-0.20765346]
 [ 0.25529982]
 [-0.28569895]
 [ 0.49571835]] 

Diff:
 [-0.00350717  0.00125709  0.04335228 -0.00679773  0.00433949  0.00195278
  0.00187143  0.00041435  0.01805953  0.01796133] 

Step  29 :

Beta:
 [[-0.44753805]
 [ 0.07031429]
 [-0.26806109]
 [-0.18441397]
 [ 0.26571817]
 [-0.22298364]
 [-0.20572251]
 [ 0.25613862]
 [-0.26696132]
 [ 0.5146396 ]] 

Diff:
 [ 0.00308212 -0.00128366 -0.04544088  0.00709182 -0.00434202 -0.00205883
 -0.00193094 -0.0008388  -0.01873763 -0.01892125] 

Step  30 :

Beta:
 [[-0.44357069]
 [ 0.06902157]
 [-0.3155959 ]
 [-0.17707416]
 [ 0.26143534]
 [-0.22520197]
 [-0.20776158]
 [ 0.25493254]
 [-0.28644152]
 [ 0.49487398]] 

Diff:
 [-0.00396737  0.00129272  0.04753481 -0.0073398   0.00428282  0.00221833
  0.00203907  0.00120608  0.01948021  0.01976561] 

Step  31 :

Beta:
 [[-0.44702679]
 [ 0.07034314]
 [-0.26590044]
 [-0.18471684]
 [ 0.2657

Step  55 :

Beta:
 [[-0.43944703]
 [ 0.07059452]
 [-0.24826908]
 [-0.18712297]
 [ 0.26581339]
 [-0.22212718]
 [-0.20532192]
 [ 0.2605232 ]
 [-0.26091412]
 [ 0.52397221]] 

Diff:
 [ 0.00719238 -0.00158256 -0.08540839  0.01189757 -0.0035999  -0.00458764
 -0.0034701  -0.00880732 -0.03189458 -0.03600686] 

Step  56 :

Beta:
 [[-0.43187768]
 [ 0.06901669]
 [-0.33410059]
 [-0.17519694]
 [ 0.26225675]
 [-0.22675633]
 [-0.20881765]
 [ 0.25161228]
 [-0.29293987]
 [ 0.48781344]] 

Diff:
 [-0.00756936  0.00157783  0.0858315  -0.01192603  0.00355664  0.00462915
  0.00349573  0.00891091  0.03202575  0.03615877] 

Step  57 :

Beta:
 [[-0.43919703]
 [ 0.07059733]
 [-0.24789048]
 [-0.18716601]
 [ 0.26579803]
 [-0.22211146]
 [-0.20531898]
 [ 0.26064379]
 [-0.26082394]
 [ 0.52415699]] 

Diff:
 [ 0.00731935 -0.00158065 -0.08621011  0.01196908 -0.00354128 -0.00464487
 -0.00349868 -0.00903151 -0.03211593 -0.03634355] 

Step  58 :

Beta:
 [[-0.43158533]
 [ 0.0690212 ]
 [-0.33442174]
 [-0.17517788]
 [ 0.2622

Step  82 :

Beta:
 [[-0.43071935]
 [ 0.06905063]
 [-0.33532464]
 [-0.17517116]
 [ 0.26248924]
 [-0.22688858]
 [-0.20888367]
 [ 0.25114859]
 [-0.29321898]
 [ 0.48737629]] 

Diff:
 [-0.00778303  0.00153709  0.08850846 -0.01206698  0.00316921  0.00483143
  0.00356447  0.00998787  0.03255861  0.03730558] 

Step  83 :

Beta:
 [[-0.43849974]
 [ 0.07058685]
 [-0.24680865]
 [-0.18723628]
 [ 0.26565321]
 [-0.22205638]
 [-0.20531954]
 [ 0.26114637]
 [-0.26066312]
 [ 0.52468549]] 

Diff:
 [ 0.00778039 -0.00153622 -0.08851599  0.01206512 -0.00316397 -0.0048322
 -0.00356413 -0.00999778 -0.03255586 -0.0373092 ] 

Step  84 :

Beta:
 [[-0.43071229]
 [ 0.06905156]
 [-0.33533101]
 [-0.17517325]
 [ 0.26249443]
 [-0.2268896 ]
 [-0.20888367]
 [ 0.25114005]
 [-0.29321673]
 [ 0.48737396]] 

Diff:
 [-0.00778745  0.00153529  0.08852236 -0.01206303  0.00315878  0.00483322
  0.00356413  0.01000632  0.03255361  0.03731153] 

Step  85 :

Beta:
 [[-0.43849816]
 [ 0.07058608]
 [-0.24680304]
 [-0.18723451]
 [ 0.26564

Step  9 :

Beta:
 [[-0.52274802]
 [ 0.07453501]
 [-0.30821609]
 [-0.19224991]
 [ 0.28072082]
 [-0.23795418]
 [-0.21928145]
 [ 0.2706508 ]
 [-0.29313144]
 [ 0.53585348]] 

Diff:
 [ 1.26403848e-05  3.02752591e-08 -9.99001420e-08 -6.89630744e-08
  1.04640956e-07 -8.39038877e-08 -7.61795214e-08  9.01855429e-08
 -9.83321303e-08  1.87970531e-07] 

Step  10 :

Beta:
 [[-0.52275244]
 [ 0.074535  ]
 [-0.30821605]
 [-0.19224988]
 [ 0.28072079]
 [-0.23795415]
 [-0.21928143]
 [ 0.27065077]
 [-0.29313141]
 [ 0.53585341]] 

Diff:
 [ 4.41987794e-06  1.05839243e-08 -3.49234681e-08 -2.41081669e-08
  3.65805686e-08 -2.93312337e-08 -2.66309660e-08  3.15272421e-08
 -3.43749319e-08  6.57110335e-08] 

Step  11 :

Beta:
 [[-0.52275398]
 [ 0.074535  ]
 [-0.30821604]
 [-0.19224988]
 [ 0.28072078]
 [-0.23795414]
 [-0.21928142]
 [ 0.27065076]
 [-0.2931314 ]
 [ 0.53585339]] 

Diff:
 [ 1.54545606e-06  3.70050583e-09 -1.22103847e-08 -8.42897454e-09
  1.27897281e-08 -1.02551294e-08 -9.31103567e-09  1.10229424e-08
 -

Step  13 :

Beta:
 [[-0.52393016]
 [ 0.0746702 ]
 [-0.30867252]
 [-0.1925526 ]
 [ 0.28118171]
 [-0.23832552]
 [-0.2196199 ]
 [ 0.27105867]
 [-0.293571  ]
 [ 0.53668888]] 

Diff:
 [ 2.95990210e-09 -1.86703125e-12  5.91401162e-12  3.45071865e-12
 -5.24007548e-12  4.31769930e-12  4.04878846e-12 -5.03359705e-12
  4.96129170e-12 -9.94213178e-12] 

Step  14 :

Beta:
 [[-0.52393016]
 [ 0.0746702 ]
 [-0.30867252]
 [-0.1925526 ]
 [ 0.28118171]
 [-0.23832552]
 [-0.2196199 ]
 [ 0.27105867]
 [-0.293571  ]
 [ 0.53668888]] 

Diff:
 [ 1.04035477e-09 -6.56248070e-13  2.07879355e-12  1.21290446e-12
 -1.84186284e-12  1.51761737e-12  1.42309737e-12 -1.76912646e-12
  1.74380150e-12 -3.49445507e-12] 

Step  15 :

Beta:
 [[-0.52393016]
 [ 0.0746702 ]
 [-0.30867252]
 [-0.1925526 ]
 [ 0.28118171]
 [-0.23832552]
 [-0.2196199 ]
 [ 0.27105867]
 [-0.293571  ]
 [ 0.53668888]] 

Diff:
 [ 3.65665699e-10 -2.30696415e-13  7.30547890e-13  4.26245306e-13
 -6.47351736e-13  5.33421248e-13  5.00195067e-13 -6.21761791e-13
 

[array([[-0.52393016],
        [ 0.0746702 ],
        [-0.30867252],
        [-0.1925526 ],
        [ 0.28118171],
        [-0.23832552],
        [-0.2196199 ],
        [ 0.27105867],
        [-0.293571  ],
        [ 0.53668888]]), array([-0.41237495,  0.4068193 ])]

In [467]:
len(out0)

10

In [27]:
# Loading required packages
from numpy.polynomial.hermite import hermgauss
from numpy.linalg import inv
import pandas as pd
import numpy as np
import scipy
import time
from scipy.stats import norm

def Pi(x, beta_0, mu):
    result = np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu))))
    if np.exp(x @ beta_0 + mu).max() == np.inf:
        return np.nan_to_num(result, nan=1)
    else:
        return result

def g(x, y, mu, beta_0, tau=1):
    g = sum(y * np.log(Pi(x, beta_0, mu)) + (1 - y) * np.log(1 - Pi(x, beta_0, mu))) \
        + np.log((np.sqrt(2 * np.pi) * tau) ** (-1) * np.exp(-mu ** 2 / (2 * tau ** 2)))
    return g

def g_b(x, y, mu, beta_0, tau=1):
    return np.sum(x * y - x * Pi(x, beta_0, mu), axis=0)

def g_u(x, y, mu, beta_0, tau=1):
    return sum(y - Pi(x, beta_0, mu)) - mu / tau ** 2

def g_uu(x, y, mu, beta_0, tau=1):
    result = np.nan_to_num(- np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu)) ** 2)), nan=0)
    return sum(result) - 1 / tau ** 2

def g_ub(x, y, mu, beta_0, tau=1):
    result = np.nan_to_num(np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu)) ** 2)), nan=0)
    return np.sum(- x * result, axis=0)

def g_bb(x, y, mu, beta_0, tau=1):
    result = 0
    for i in range(len(y)):
        result += -np.asarray(x[i].reshape(x.shape[1], 1) @ x[i].reshape(1, x.shape[1]) \
                              * np.nan_to_num((np.exp(x[i] @ beta_0 + mu) / (1 + np.exp(x[i] @ beta_0 + mu)) ** 2),
                                              nan=0))
    return result

def g_uuu(x, y, mu, beta_0, tau=1):
    return sum(- np.nan_to_num(np.asarray((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1) \
                                           / (1 + np.exp(x @ beta_0 + mu)) ** 3)), nan=0))

def g_uub(x, y, mu, beta_0, tau=1):
    return np.sum(- x * np.nan_to_num(np.asarray((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1) \
                                                  / (1 + np.exp(x @ beta_0 + mu)) ** 3)), nan=0), axis=0)

def g_ubb(x, y, mu, beta_0, tau=1):
    result = 0
    for i in range(len(y)):
        result += -np.asarray(x[i].reshape(x.shape[1], 1) @ x[i].reshape(1, x.shape[1]) \
                              * np.nan_to_num((np.exp(x[i] @ beta_0 + mu) * (np.exp(x[i] @ beta_0 + mu) - 1) \
                                               / (1 + np.exp(x[i] @ beta_0 + mu)) ** 3), nan=0))
    return result

def g_uuuu(x, y, mu, beta_0, tau=1):
    result = sum(- np.nan_to_num((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1) \
                                  / (1 + np.exp(x @ beta_0 + mu)) ** 3), nan=0) \
                 + np.nan_to_num((3 * np.exp(2 * (x @ beta_0 + mu)) * (np.exp(x @ beta_0 + mu) - 1) \
                                  / (1 + np.exp(x @ beta_0 + mu)) ** 4), nan=0) \
                 - np.nan_to_num((np.exp(2 * (x @ beta_0 + mu)) / (1 + np.exp(x @ beta_0 + mu)) ** 3), nan=0) \
                 )
    return result

def g_uuub(x, y, mu, beta_0, tau=1):
    result = np.sum(- x * np.asarray(np.nan_to_num((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1) \
                                                    / (1 + np.exp(x @ beta_0 + mu)) ** 3), nan=0) \
                                     + np.nan_to_num(
        (3 * np.exp(2 * (x @ beta_0 + mu)) * (np.exp(x @ beta_0 + mu) - 1) \
         / (1 + np.exp(x @ beta_0 + mu)) ** 4), nan=0) \
                                     - np.nan_to_num(
        (np.exp(2 * (x @ beta_0 + mu)) / (1 + np.exp(x @ beta_0 + mu)) ** 3), nan=0))
                    , axis=0)
    return result

def g_uubb(x, y, mu, beta_0, tau=1):
    result = 0
    for i in range(len(y)):
        result += -np.asarray(x[i].reshape(x.shape[1], 1) @ x[i].reshape(1, x.shape[1]) \
                              * (np.nan_to_num((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1) \
                                                / (1 + np.exp(x @ beta_0 + mu)) ** 3), nan=0) \
                                 + np.nan_to_num((3 * np.exp(2 * (x @ beta_0 + mu)) * (np.exp(x @ beta_0 + mu) - 1) \
                                                  / (1 + np.exp(x @ beta_0 + mu)) ** 4), nan=0) \
                                 - np.nan_to_num(
                    (np.exp(2 * (x @ beta_0 + mu)) / (1 + np.exp(x @ beta_0 + mu)) ** 3), nan=0)))

    return result

def omega(x, y, mu, beta_0, tau=1):
    return np.sqrt(-1 / g_uu(x, y, mu, beta_0))

def omega_b(x, y, mu, beta_0, tau=1):
    return 0.5 * omega(x, y, mu, beta_0, tau) ** 3 * (g_uuu(x, y, mu, beta_0, tau) * mu_b(x, y, mu, beta_0, tau) \
                                                      + g_uub(x, y, mu, beta_0, tau))

def mu_b(x, y, mu, beta_0, tau=1):
    return omega(x, y, mu, beta_0, tau) ** 2 * g_ub(x, y, mu, beta_0, tau)

def mu_bb(x, y, mu, beta_0, tau=1):
    result = omega(x, y, mu, beta_0, tau) ** 2 * (mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1], 1) \
                                                  @ mu_b(x, y, mu, beta_0, tau).reshape(1, x.shape[1]) \
                                                  * g_uuu(x, y, mu, beta_0, tau) \
                                                  + 2 * mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1], 1) \
                                                  @ g_uub(x, y, mu, beta_0, tau).reshape(1, x.shape[1]) \
                                                  + g_ubb(x, y, mu, beta_0, tau))
    return result

def omega_bb(x, y, mu, beta_0, tau=1):
    result = 3 / 4 * omega(x, y, mu, beta_0, tau) ** 5 * (mu_b(x, y, mu, beta_0, tau) * g_uuu(x, y, mu, beta_0, tau) \
                                                          + g_uub(x, y, mu, beta_0, tau)).reshape(x.shape[1], 1) \
             @ (mu_b(x, y, mu, beta_0, tau) * g_uuu(x, y, mu, beta_0, tau) \
                + g_uub(x, y, mu, beta_0, tau)).reshape(1, x.shape[1]) + 1 / 2 * omega(x, y, mu, beta_0, tau) ** 3 \
             * (mu_bb(x, y, mu, beta_0, tau) * g_uuu(x, y, mu, beta_0, tau) \
                + (mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1], 1) \
                   @ mu_b(x, y, mu, beta_0, tau).reshape(1, x.shape[1]) * g_uuuu(x, y, mu, beta_0, tau)) \
                + mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1], 1) @ g_uuub(x, y, mu, beta_0, tau).reshape(1,
                                                                                                             x.shape[
                                                                                                                 1]) \
                + mu_bb(x, y, mu, beta_0, tau)
                )
    return result

def l_1(x, y, mu, beta_0, tau=1):
    l1 = omega_b(x, y, mu, beta_0, tau) / omega(x, y, mu, beta_0, tau) \
         + g_u(x, y, mu, beta_0, tau) * mu_b(x, y, mu, beta_0, tau) + g_b(x, y, mu, beta_0, tau)
    return l1

def l_2(x, y, mu, beta_0, tau=1):
    l2 = omega(x, y, mu, beta_0, tau) ** (-2) * (omega_bb(x, y, mu, beta_0, tau) * omega(x, y, mu, beta_0, tau) \
                                                 - omega_b(x, y, mu, beta_0, tau).reshape(x.shape[1], 1) \
                                                 @ omega_b(x, y, mu, beta_0, tau).reshape(1, x.shape[1])) \
         + mu_bb(x, y, mu, beta_0, tau) * g_u(x, y, mu, beta_0, tau) + mu_b(x, y, mu, beta_0, tau).reshape(
        x.shape[1], 1) \
         @ (mu_b(x, y, mu, beta_0, tau) * g_uu(x, y, mu, beta_0, tau) + g_ub(x, y, mu, beta_0, tau)).reshape(1,
                                                                                                             x.shape[
                                                                                                                 1]) \
         + mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1], 1) @ g_ub(x, y, mu, beta_0, tau).reshape(1, x.shape[1]) \
         + g_bb(x, y, mu, beta_0, tau)
    return l2

def max_mu(x, y, mu, beta_0, tau=1, max_iter=100):
    for step in range(max_iter):
        #         print('Step: ', step, '\n')
        mu_new = mu - g_u(x, y, mu, beta_0, tau) / g_uu(x, y, mu, beta_0, tau)
        diff = mu_new - mu
        #         print(diff)
        if np.abs(diff) < 10 ** (-10):
            #             print(mu)
            break;
        mu = mu_new
    return mu

# Definitions for LA
def LA(X, y):

    beta = np.repeat(0, 10).reshape(10, 1)
    mu = np.repeat(0.1, len(y))
    tau = 1

    start_time = time.time()
    print('Initial beta:', beta, "\n")
    for step_mu in range(3):
        for i in range(len(mu)):
            mu[i] = max_mu(X[i], y[i], mu[i], beta, tau)
        for step in range(100):
            l1 = 0
            l2 = 0
            for i in range(len(mu)):
                l1 += l_1(X[i], y[i], mu[i], beta, tau)
                l2 += l_2(X[i], y[i], mu[i], beta, tau)
            delta = l1 @ inv(l2)
            new_beta = beta - delta.reshape(10, 1)
            if max(np.abs(delta)) < 10 ** (-10):
                break;
            beta = new_beta
            if True in np.isnan(beta):
                break;
            print('Step ', step + 1, ':\n')
            print('Beta:\n', beta, '\n')
            print('Diff:\n', delta, '\n')
        if True in np.isnan(beta):
            break;
    print('Beta:\n', beta, '\n')
    print("--- %s seconds ---" % (time.time() - start_time))
    return [beta, mu]

def output(X, beta, true_beta):
    X = np.concatenate(X)

    V = np.diagflat(Pi(X, beta, 0) * (1 - Pi(X, beta, 0)))

    SE = np.sqrt(np.diag(inv(np.transpose(X) @ V @ X))).reshape(10,1)

    Z = beta/SE

    P = 2 * norm.cdf(-1 * np.abs(Z))

    CI_025  = beta - 1.959964 * SE
    CI_975  = beta + 1.959964 * SE

    df = pd.DataFrame({'Truth': np.transpose(true_beta)[0], 'Coef': np.transpose(beta)[0], 'Std.Err': np.transpose(SE)[0],
                       'z': np.transpose(Z)[0], 'P-value': np.transpose(P)[0],
                       '[0.025': np.transpose(CI_025)[0], '0.975]': np.transpose(CI_975)[0]},
                      index = var_name)
    return df


##################### Simulation with Penn ############################
import os
truth = np.array([-1.5,0.1,-0.5,-0.3,0.4,-0.2,-0.25,0.35,-0.1,0.5]).reshape(10, 1)
var_name = []
for i in range(10):
    var_name += ['X' + str(i+1)]

# Setting 1
print('======================\n Here starts Setting 1 \n======================\n\n\n')

file_dir = '../Simulation_data_GLMM/Setting_1/'
file_names = os.listdir(file_dir)
for i in range(len(file_names)):
    start_time = time.time()
    df = pd.read_csv(file_dir + file_names[i], index_col=0)
    # Load data
    data1 = np.array(df[df['Site_ID'] == 1][var_name])
    data2 = np.array(df[df['Site_ID'] == 2][var_name])
    data = [data1, data2]
    out1 = np.array(df[df['Site_ID'] == 1]['y']).reshape(500,1)
    out2 = np.array(df[df['Site_ID'] == 2]['y']).reshape(500,1)
    out = [out1, out2]
    # Simulations
    [beta, mu] = LA(data, out)
    output(data, beta, truth).to_csv('../Simulation_data_GLMM/Result/Setting_1_' +
                                          file_names[i][44:], header = True)
    print('======\n File:\n', file_names[i], '\n Completed!\n')
    print("--- %s seconds ---" % (time.time() - start_time))

 Here starts Setting 1 



Initial beta: [[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]] 

Step  1 :

Beta:
 [[-0.25278673]
 [ 0.11248096]
 [-0.42087669]
 [-0.50730788]
 [ 0.2297902 ]
 [-0.18183595]
 [-0.19928937]
 [ 0.09249199]
 [-0.16234769]
 [ 0.39603518]] 

Diff:
 [ 0.25278673 -0.11248096  0.42087669  0.50730788 -0.2297902   0.18183595
  0.19928937 -0.09249199  0.16234769 -0.39603518] 

Step  2 :

Beta:
 [[-0.39049015]
 [ 0.09544239]
 [-0.46450534]
 [-0.46144605]
 [ 0.21908947]
 [-0.18379527]
 [-0.19469118]
 [ 0.10001121]
 [-0.15992504]
 [ 0.36657051]] 

Diff:
 [ 0.13770343  0.01703857  0.04362865 -0.04586183  0.01070073  0.00195931
 -0.00459819 -0.00751921 -0.00242265  0.02946466] 

Step  3 :

Beta:
 [[-0.47448576]
 [ 0.11816304]
 [-0.36035122]
 [-0.51935546]
 [ 0.22605943]
 [-0.17086563]
 [-0.19065116]
 [ 0.08828856]
 [-0.15489545]
 [ 0.39396253]] 

Diff:
 [ 0.08399561 -0.02272065 -0.10415412  0.0579094  -0.00696997 -0.01292963
 -0.00404002  0.01172265 -0.00502959 -0.02739201]

Step  28 :

Beta:
 [[ 0.54550704]
 [ 0.12435925]
 [-0.21405487]
 [-0.61124657]
 [ 0.26255464]
 [-0.20325345]
 [-0.18781282]
 [ 0.2207246 ]
 [-0.1761106 ]
 [ 0.39533321]] 

Diff:
 [-8.22173360e-01 -2.97302879e-04 -4.56880989e-01  5.85490755e-02
  1.64175350e-03 -2.27603403e-02 -7.39469857e-02 -1.67725724e-01
 -2.92412255e-02  1.02179147e-01] 

Step  29 :

Beta:
 [[-0.15946235]
 [ 0.12095069]
 [-0.44288788]
 [-0.56260206]
 [ 0.25593282]
 [-0.20691451]
 [-0.21636387]
 [ 0.13086132]
 [-0.18146715]
 [ 0.42745088]] 

Diff:
 [ 0.70496939  0.00340856  0.22883301 -0.04864451  0.00662182  0.00366106
  0.02855105  0.08986328  0.00535655 -0.03211767] 

Step  30 :

Beta:
 [[-0.1146944 ]
 [ 0.09950951]
 [-0.46906817]
 [-0.45986092]
 [ 0.21466747]
 [-0.1777182 ]
 [-0.19648703]
 [ 0.07907226]
 [-0.1585537 ]
 [ 0.37879866]] 

Diff:
 [-0.04476795  0.02144118  0.02618028 -0.10274114  0.04126534 -0.02919631
 -0.01987684  0.05178906 -0.02291345  0.04865222] 

Step  31 :

Beta:
 [[-0.4804492 ]
 [ 0.10712523

Step  56 :

Beta:
 [[-0.50150036]
 [ 0.10296979]
 [-0.45936342]
 [-0.47646644]
 [ 0.21961968]
 [-0.18028027]
 [-0.1983174 ]
 [ 0.08421677]
 [-0.1619193 ]
 [ 0.38353958]] 

Diff:
 [ 0.39927459  0.00150741  0.05306091 -0.01466031  0.00743376 -0.00156015
  0.00705101  0.02623065  0.00537597 -0.01068765] 

Step  57 :

Beta:
 [[-0.40208754]
 [ 0.10962821]
 [-0.34889086]
 [-0.52137777]
 [ 0.24315922]
 [-0.18781294]
 [-0.18704804]
 [ 0.14312232]
 [-0.15452351]
 [ 0.36760328]] 

Diff:
 [-0.09941282 -0.00665842 -0.11047256  0.04491133 -0.02353954  0.00753268
 -0.01126936 -0.05890554 -0.00739578  0.0159363 ] 

Step  58 :

Beta:
 [[-0.47267308]
 [ 0.08889725]
 [-0.63899436]
 [-0.37813201]
 [ 0.16856163]
 [-0.16188759]
 [-0.21318975]
 [-0.02326631]
 [-0.16959756]
 [ 0.40387611]] 

Diff:
 [ 0.07058554  0.02073097  0.2901035  -0.14324576  0.07459758 -0.02592535
  0.0261417   0.16638863  0.01507405 -0.03627283] 

Step  59 :

Beta:
 [[-0.11473476]
 [ 0.18691688]
 [ 0.25002003]
 [-0.88559684]
 [ 0.4480

KeyboardInterrupt: 